In [1]:
!nvidia-smi

Sun Oct  3 06:09:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [3]:
!python --version

Python 3.7.12


In [4]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 636 kB 58.2 MB/s 
     |████████████████████████████████| 895 kB 56.1 MB/s 
     |████████████████████████████████| 3.3 MB 81.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.1 MB/s 


# Get data from this link:- (https://drive.google.com/file/d/1bJQ240quwpbkSHlt0h71uebtR5obWrVg/view?usp=sharing)

In [6]:
import pandas as pd
from google.colab import files
uploaded=files.upload()

Saving train.csv to train.csv


# New Section

In [7]:
df= pd.read_csv('train.csv')
df.columns=['A','B','C']

possible_labels = df.C.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['label'] = df.C.replace(label_dict)
df.head()

,A,B,C,label
0,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help,0
1,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings,1
2,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great,2
3,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link,3
4,BOS Understood EOS,O O O,great,2


In [8]:
df['label']

0       0
1       1
2       2
3       3
4       2
       ..
3508    0
3509    1
3510    5
3511    0
3512    0
Name: label, Length: 3513, dtype: int64

In [9]:
n = len(pd.unique(df['C'])) #number of unique labels
print(n)

151


In [10]:
# Load the dataset into a pandas dataframe.
      
      #df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])


# Report the number of sentences.
print('Number of training sentences:')
print(df.shape[0])
# Display 10 random rows from the data.
df.sample(10)

Number of training sentences:
3513


,A,B,C,label
336,BOS <ss type=hi>(wave)</ss> EOS,O O O O,Greetings,1
1456,BOS these thinngs are not covered yet sir...ne...,O O O O O O O O O O,course_contents,84
1398,BOS Hi EOS,O O O,Greetings,1
2897,BOS I got this error while trying to install t...,O O O O O O O O O O O,tensorflow_installation,63
2240,BOS vishnuderkar@gmail.com EOS,O O O,user_email,44
1662,BOS How to register to that denied course? Ple...,O O O O O O O O O O O O,support_ticket_generate,10
3042,BOS Sir/Madam I heard for Machine Learning Cou...,O O O O O O O O O O O O O O O O O O O O O,need_help,0
1490,BOS Hey EOS,O O O,Greetings,1
1781,BOS I have not received the Tableau and powerB...,O O O O O O O O O O O O O,power-bi_tableau_dashboard,20
1008,BOS how to do this above step will u please gu...,O O O O O O O O O O O O O,need_help,0


In [11]:
sentences = df.A.values
labels = df.label.values

In [12]:
from transformers import AlbertTokenizer

# Load the ALBERT tokenizer.
print('Loading ALBERT tokenizer...')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

Loading ALBERT tokenizer...


Downloading:   0%|          | 0.00/742k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

In [13]:
tokenizer

PreTrainedTokenizer(name_or_path='albert-base-v2', vocab_size=30000, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [14]:
pwd()

'/content'

In [15]:
ls

Albert-20211003T055833Z-001.zip  sample_data/  train.csv


In [16]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    print(input_ids)
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

[2, 11054, 784, 1448, 184, 129, 32, 8402, 13, 60, 60, 13, 3894, 18, 3]
[2, 11054, 10975, 927, 25, 80, 1276, 13, 3894, 18, 3]
[2, 11054, 31, 277, 9696, 6738, 108, 1448, 23, 397, 20, 16495, 9, 9, 31, 794, 19, 13, 12985, 17, 32, 577, 13, 3894, 18, 3]
[2, 11054, 2247, 1181, 55, 29, 14, 3508, 20, 20059, 718, 9, 13, 3894, 18, 3]
[2, 11054, 4014, 13, 3894, 18, 3]
[2, 11054, 13, 38, 1251, 13, 3894, 18, 3]
[2, 11054, 86, 1838, 114, 4344, 640, 62, 98, 31, 417, 20, 143, 13, 3894, 18, 3]
[2, 11054, 927, 186, 2538, 17419, 160, 3309, 20, 9059, 48, 7019, 60, 13, 3894, 18, 3]
[2, 11054, 13, 8157, 9, 2195, 5555, 9, 23714, 5, 2558, 1569, 14574, 18, 73, 839, 192, 217, 1569, 14574, 18, 73, 15, 279, 14574, 18, 73, 839, 192, 293, 1569, 14574, 18, 73, 500, 15, 165, 15, 306, 3726, 2558, 387, 9, 3283, 3165, 25748, 2409, 24366, 3902, 3412, 15, 713, 9, 10609, 4060, 264, 23574, 23954, 24570, 1543, 500, 6, 2558, 387, 500, 13, 3894, 18, 3]
[2, 11054, 19, 1809, 43, 18, 5935, 203, 897, 53, 13, 3894, 18, 3]
[2, 11054,

Token indices sequence length is longer than the specified maximum sequence length for this model (634 > 512). Running this sequence through the model will result in indexing errors


[2, 11054, 13, 1, 3091, 5739, 1001, 3726, 7292, 69, 7051, 3726, 645, 1340, 3726, 3384, 546, 2602, 2887, 8, 2409, 43, 465, 8, 300, 6887, 135, 8, 3804, 518, 410, 8, 379, 43, 264, 62, 2851, 2032, 43, 2460, 135, 1, 3091, 3270, 3726, 5928, 45, 93, 6031, 1323, 8707, 1, 7259, 1, 93, 6031, 21, 4973, 6423, 1, 118, 7259, 1, 10852, 857, 1, 379, 9, 2887, 300, 1, 118, 10852, 857, 1, 118, 3091, 1, 3091, 3270, 3726, 5928, 45, 9, 10220, 9, 3624, 4048, 58, 2287, 12425, 3776, 2918, 4268, 1, 7259, 1, 24168, 1, 108, 5630, 218, 9, 3622, 1, 118, 7259, 1, 10852, 857, 1, 379, 9, 2887, 300, 1, 118, 10852, 857, 1, 118, 3091, 1, 118, 3091, 5739, 1, 13, 3894, 18, 3]
[2, 11054, 927, 31, 794, 20, 17617, 27, 36, 9266, 188, 37, 14, 1422, 768, 14, 205, 829, 13, 3894, 18, 3]
[2, 11054, 31, 589, 1017, 7019, 133, 4721, 12613, 13, 3894, 18, 3]
[2, 11054, 927, 19, 48, 293, 98, 11374, 137, 17, 351, 845, 13, 3894, 18, 3]
[2, 11054, 589, 1017, 13, 3894, 18, 3]
[2, 11054, 13, 1, 3091, 5739, 1001, 3726, 10726, 7051, 3726, 645, 

In [17]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 100,           # Pad & truncate all sentences. Just in case there are some longer test sentences, I’ll set the maximum length to 64.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                        truncation=True
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  BOS above problem how will it solve ?? EOS
Token IDs: tensor([    2, 11054,   784,  1448,   184,   129,    32,  8402,    13,    60,
           60,    13,  3894,    18,     3,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])


In [ ]:
attention_masks[4]

tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [ ]:
input_ids[4]

tensor([    2, 11054,  4014,    13,  3894,    18,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [ ]:
input_ids

tensor([[    2, 11054,   784,  ...,     0,     0,     0],
        [    2, 11054, 10975,  ...,     0,     0,     0],
        [    2, 11054,    31,  ...,     0,     0,     0],
        ...,
        [    2, 11054,   130,  ...,     0,     0,     0],
        [    2, 11054,    31,  ...,     0,     0,     0],
        [    2, 11054,  3893,  ...,     0,     0,     0]])

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

3,161 training samples
  352 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import     AlbertForSequenceClassification, AdamW, AlbertConfig

# Load AlertForSequenceClassification, the pretrained ALBERT model with a single 
# linear classification layer on top. 
model = AlbertForSequenceClassification.from_pretrained(
    "albert-large-v2", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels =151 , # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    return_dict=False
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at albert-large-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-large-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You s

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=1024, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_featur

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The ALBERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The ALBERT model has 27 different named parameters.

==== Embedding Layer ====

albert.embeddings.word_embeddings.weight                (30000, 128)
albert.embeddings.position_embeddings.weight              (512, 128)
albert.embeddings.token_type_embeddings.weight              (2, 128)
albert.embeddings.LayerNorm.weight                            (128,)
albert.embeddings.LayerNorm.bias                              (128,)

==== First Transformer ====

albert.encoder.embedding_hidden_mapping_in.weight        (1024, 128)
albert.encoder.embedding_hidden_mapping_in.bias              (1024,)
albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight      (1024,)
albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias      (1024,)
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight (1024, 1024)
albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.bias      (1024,)
albert.encoder.albert_layer_groups.0.albert

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 4 ========
Training...
  Batch    40  of     99.    Elapsed: 0:00:22.
  Batch    80  of     99.    Elapsed: 0:00:45.

  Average training loss: 3.35
  Training epcoh took: 0:00:55

Running Validation...
  Accuracy: 0.48
  Validation Loss: 2.59
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...
  Batch    40  of     99.    Elapsed: 0:00:22.
  Batch    80  of     99.    Elapsed: 0:00:45.

  Average training loss: 2.31
  Training epcoh took: 0:00:55

Running Validation...
  Accuracy: 0.53
  Validation Loss: 2.30
  Validation took: 0:00:02

======== Epoch 3 / 4 ========
Training...
  Batch    40  of     99.    Elapsed: 0:00:22.
  Batch    80  of     99.    Elapsed: 0:00:44.

  Average training loss: 2.04
  Training epcoh took: 0:00:55

Running Validation...
  Accuracy: 0.55
  Validation Loss: 2.19
  Validation took: 0:00:02

======== Epoch 4 / 4 ========
Training...
  Batch    40  of     99.    Elapsed: 0:00:22.
  Batch    80  of     99.    Elapsed: 0:

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,3.35,2.59,0.48,0:00:55,0:00:02
2,2.31,2.30,0.53,0:00:55,0:00:02
3,2.04,2.19,0.55,0:00:55,0:00:02
4,1.87,2.15,0.55,0:00:55,0:00:02


## Test

# Get data from this link:- (https://drive.google.com/file/d/1-Tg3Znr6c1_bwyHaUihUlT2JBH6QkeNG/view?usp=sharing)

In [ ]:
import pandas as pd
from google.colab import files
uploaded=files.upload()

Saving train.csv to train (1).csv


In [ ]:
# df = pd.read_csv('test2.csv')

# possible_labels = df.label.unique()

# label_dict = {}
# for index, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = index

# df['labeled'] = df.label.replace(label_dict)
# df.head()

In [ ]:
import pandas as pd

#Load the dataset into a pandas dataframe.
df = pd.read_csv('train.csv', header=None)
df.columns = ['sentence', 'Unnamed: 1', 'label']

possible_labels = df.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['labeled'] = df.label.replace(label_dict)
df.head()

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

Number of test sentences: 3,514



In [ ]:
# Create sentence and label lists
sentences = df.sentence.values
labels = df.labeled.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
df = pd.read_csv('train.csv', header=None)
df.columns = ['sentence', 'Unnamed: 1', 'label']
df

,sentence,Unnamed: 1,label
0,BOS Sir I created a new env with python 3.7 bu...,O O O O O O O O O O O O O O O O O O O O O O O ...,need_help
1,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help
2,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings
3,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great
4,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link
...,...,...,...
3509,BOS it is not one to one resume but some advic...,O O O O O O O O O O O O O O O O,need_help
3510,BOS hi good noon EOS,O O O O O,Greetings
3511,BOS now what will year contain EOS,O O O O O O O,code_error
3512,BOS I have extracted it but didn&apos;t found ...,O O O O O O O O O O,need_help


In [ ]:
# df = pd.read_csv('test2.csv')
# df

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 3,514 test sentences...
    DONE.


In [ ]:
predictions

[array([[-0.7205367 , -0.40204504,  1.2971681 , ..., -0.7275311 ,
          1.0460682 ,  0.8605784 ],
        [ 0.09857273, -0.49066323,  0.4911199 , ..., -0.7831502 ,
          0.5308529 ,  0.7307584 ],
        [-0.07101091, -0.54117304,  0.46358767, ..., -0.66856575,
          0.51124114,  0.62275195],
        ...,
        [-0.09330106, -0.5105448 ,  0.6518375 , ..., -0.6664418 ,
          0.5031544 ,  0.7623328 ],
        [-0.05565155, -0.49264848,  0.63554585, ..., -0.7435494 ,
          0.653846  ,  0.74536705],
        [ 0.1447613 , -0.42320842,  0.35067487, ..., -0.5017137 ,
          0.67172897,  0.5724275 ]], dtype=float32),
 array([[ 0.07328643, -0.4327707 ,  0.12453593, ..., -0.5722867 ,
          0.4267402 ,  0.51982474],
        [-0.04648045, -0.47331768,  0.56031317, ..., -0.69753045,
          0.5167936 ,  0.67234486],
        [ 0.00729236, -0.21505621,  0.00210112, ..., -0.79348195,
         -0.12395727,  0.71145046],
        ...,
        [ 0.20656389, -0.40290397,  0.0

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/spiece.model',
 './model_save/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
